In [1]:
import time
import numpy as np
import pandas as pd

from pycaret.datasets import get_data
from pycaret.classification import *

In [2]:
data = get_data('juice', verbose=False)

In [3]:
y = ["a" if i else "b" for i in data.iloc[:, -1]]

In [4]:
exp = setup(data=data, test_data=data.iloc[:10, :], target=-1, session_id=123)

Alert! from version 0.1.39, fter importing, you must do '%matplotlib inline' to display charts in Jupyter Notebooks.
    AV = AutoViz_Class()
    dfte = AV.AutoViz(filename, sep=',', depVar='', dfte=None, header=0, verbose=0, lowess=False,
               chart_format='svg',max_rows_analyzed=150000,max_cols_analyzed=30, save_plot_dir=None)
Note: verbose=0 or 1 generates charts and displays them in your local Jupyter notebook.
      verbose=2 does not display plots but saves them in AutoViz_Plots folder in local machine.
Updated: chart_format='bokeh' generates and displays charts in your local Jupyter notebook.
      chart_format='server' generates and displays charts in the browser - one tab for each chart.
      chart_format='html' silently saves charts HTML format - they are also interactive!


`scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.


,Description,Value
0,Session id,123
1,Target,STORE
2,Target type,Multiclass
3,Original data shape,"(1080, 19)"
4,Transformed data shape,"(1080, 19)"
5,Transformed train set shape,"(1070, 19)"
6,Transformed test set shape,"(10, 19)"
7,Ordinal features,2
8,Numeric features,16
9,Categorical features,2
